# 1. 讀取檔案

In [73]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# 讚嘆 GPT(雖然是 Gemini)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# 使用 pandas 的 read_csv 函式讀取訓練資料
df = pd.read_csv("data/train.csv")
median_imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()

In [74]:
def preprocess(df : pd.DataFrame, test=False) -> tuple[pd.DataFrame, pd.DataFrame]:
    filled_features = df.select_dtypes('number').columns.drop('mpg', errors='ignore')
    df_clean = df.copy()

    if not test: df_clean[filled_features] = median_imputer.fit_transform(df_clean[filled_features])
    else: df_clean[filled_features] = median_imputer.transform(df_clean[filled_features])
    
    df_final = pd.get_dummies(
        df_clean, columns=['origin'], prefix='origin', 
        drop_first=True, dtype=int
    )

    filled_features = df.select_dtypes('number').columns.drop('mpg', errors='ignore')
    features = list(filled_features) + ['origin_japan', 'origin_usa']
    X = df_final[features].copy()

    if not test: X[filled_features] = scaler.fit_transform(X[filled_features])
    else: X[filled_features] = scaler.transform(X[filled_features])

    if test: return X, None

    target = 'mpg'
    Y = df_final[target]

    return X, Y


# 2.資料預處理

In [75]:
# 直接刪除任何含有缺失值的整行資料
# HINT：有更好的預處理填補方式嗎？
df_clean = df.dropna()

# 3.特徵工程

In [76]:
# # HINT：這串列遺漏了部分特徵，另外，有方法額外加上新的特徵嗎？
# df_final = pd.get_dummies(
#     df_clean, 
#     columns=['origin'], 
#     prefix='origin', 
#     drop_first=True,
#     dtype=int
# )

# # print(df_final.head(5))

# # 定義要用來預測的特徵欄位
# features = [
#     'weight', 'acceleration', 'model_year', 
#     'cylinders', 'displacement', 'horsepower', 
#     'origin_japan', 'origin_usa'
# ]

# # 定義我們要預測的目標欄位
# target = 'mpg'

# 從乾淨的資料中選取 X 和 y
X, Y = preprocess(df)

# 4. 訓練模型

In [77]:
# 分割訓練集與測試集 (用於本地評估模型好壞)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 建立並訓練模型
model = LinearRegression()
model.fit(X_train, y_train)

# 進行預測與評估
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

print("模型已訓練完成！")
print(f"訓練誤差 (Train RMSE): {train_rmse:.4f} MPG")
print(f"測試誤差 (Test RMSE):  {test_rmse:.4f} MPG")
print("\n--- 模型學到的關係 ---")

# 各個特徵的權重
features = [
    'weight', 'acceleration', 'model_year', 
    'cylinders', 'displacement', 'horsepower', 
    'origin_japan', 'origin_usa'
]
for feature, coef in zip(features, model.coef_):
    print(f"特徵 '{feature}' 的權重: {coef:.4f}")

模型已訓練完成！
訓練誤差 (Train RMSE): 3.2941 MPG
測試誤差 (Test RMSE):  3.6265 MPG

--- 模型學到的關係 ---
特徵 'weight' 的權重: -6.2521
特徵 'acceleration' 的權重: -0.2022
特徵 'model_year' 的權重: 1.7239
特徵 'cylinders' 的權重: -0.9050
特徵 'displacement' 的權重: -5.5083
特徵 'horsepower' 的權重: -0.0669
特徵 'origin_japan' 的權重: 9.2462
特徵 'origin_usa' 的權重: -0.9362


### 初始版

訓練誤差 (Train RMSE): 3.3241 MPG

測試誤差 (Test RMSE):  3.6002 MPG

*這裡中位數設定有錯，會看到測試集

### GPT 通靈一波

訓練誤差 (Train RMSE): 3.2922 MPG

測試誤差 (Test RMSE):  3.6340 MPG

### 刪掉通靈的部分(留下標準化)

訓練誤差 (Train RMSE): 3.2922 MPG

測試誤差 (Test RMSE):  3.6340 MPG

\*不知為啥沒變

### 換成填眾數

訓練誤差 (Train RMSE): 3.2919 MPG

測試誤差 (Test RMSE):  3.6353 MPG

### 換成平均

訓練誤差 (Train RMSE): 3.2941 MPG

測試誤差 (Test RMSE):  3.6265 MPG

*神奇，Over fit?

# 5.輸出提交檔案

In [78]:
# 讀取需要進行預測的測試檔案 test.csv
df_test = pd.read_csv("data/test.csv")

# 對測試資料進行預處理
# HINT：如果前面使用了其他的預處理方式，這邊要如何修改？
X, Y = preprocess(df_test, True)

# 使用訓練好的模型，對測試資料進行預測
predictions = model.predict(X)

# 建立一個新的 DataFrame
submission_df = pd.DataFrame({'Id':df_test['id'], 'mpg': predictions})

# 保存為 submission.csv
submission_df.to_csv('submission.csv', index=False)
print("提交文件 'submission.csv' 已成功生成！")

提交文件 'submission.csv' 已成功生成！


# 6. 報告

姓名：__許博智__ 學號：_114403549_

第一部分：準確度分數 (Accuracy Scores) (1分)  
我的準確度分數：__3.6265__  

第二部分：我的實驗記錄 (My Experiment Log) (3分)  
請記錄你做了哪些嘗試來提升分數，至少記錄兩次不同的嘗試。  
【實驗 1】  
    我做的修改：__填上中位數、做 one hot__  
    結果與觀察 (分數變化、心得等)：__之前學得都忘光了 :D__  
    該次實驗分數： __3.6002__  
【實驗 2】  
    我做的修改：__(去找了 GPT)發現之前寫的中位數會吃到測試集的資料，好像不太好，調了回來。然後把資料標準化__  
    結果與觀察 (分數變化、心得等)：__GPT 真厲害 :D，分數下降了，但應該是因為之前會看到測試集的原因？__  
    該次實驗分數： __3.6340__  
【實驗 3】  
    我做的修改：__把填數值的方式改成平均和眾數試試看__  
    結果與觀察 (分數變化、心得等)：__平均不知道為甚麼比較低？怪怪的，但總之他分數最好看__  
    該次實驗分數： __3.6265__  

第三部分：總結與心得 (Conclusion & Reflection) (2分)  
請撰寫一段約 50-100 字的心得總結。內容需包含：  
(1) 你認為本次實驗中，提升準確率最有效的修改是什麼。  
(2) 這次不斷嘗試與修正的過程，帶給你最大的學習與啟發。  
內容：

~~AI 真的是在通靈~~，還是有很多步驟容易忘記和做錯，還需要多多練習。把中位數調回來後有點糾結到底有沒有錯，也不太確定，畢竟上課教的方法也和這個不太一樣。但總之弄出來了，還算能用？不太確定到底要調到多少才算好，所以就先這樣吧，之後的優化要花一點時間和腦子了，還不一定有用 .w.。還是多燒香說不定比較有救，好玩，有空再回來玩，要去讀微積分了 :P。